In [48]:
import numpy as np
import pandas as pd
import category_encoders as ce
import sys, os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import OneHotEncoder

In [49]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from file_handler import FileHandler
from df_selector import *
from df_cleaner import *
from df_visualizer import *

In [50]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

# Reading Data

In [51]:
file_handler = FileHandler()

In [52]:
# reading the store csv file

missing_values=["n/a", "na", "undefined"]
df = pd.read_csv("../data/data.csv", na_values=missing_values)
            
df.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


# General Statistics

In [53]:
# number of elements in the df
df.size

1530592

In [54]:
# rows and columns in the df
df.shape

(95662, 16)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TransactionId         95662 non-null  object 
 1   BatchId               95662 non-null  object 
 2   AccountId             95662 non-null  object 
 3   SubscriptionId        95662 non-null  object 
 4   CustomerId            95662 non-null  object 
 5   CurrencyCode          95662 non-null  object 
 6   CountryCode           95662 non-null  int64  
 7   ProviderId            95662 non-null  object 
 8   ProductId             95662 non-null  object 
 9   ProductCategory       95662 non-null  object 
 10  ChannelId             95662 non-null  object 
 11  Amount                95662 non-null  float64
 12  Value                 95662 non-null  int64  
 13  TransactionStartTime  95662 non-null  object 
 14  PricingStrategy       95662 non-null  int64  
 15  FraudResult        

# Missing Values

In [56]:
percent_missing_values(df)

The dataset contains 0.0 % missing values.


# Data Types

In [57]:
#check if there are columns with mixed data types.
show_cols_mixed_dtypes(df)

None of the columns contain mixed types.


In [58]:
df.dtypes

TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object

In [59]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [60]:
# get the columns with object data type
string_columns = df.select_dtypes(include='object').columns.tolist()
string_columns

['TransactionId',
 'BatchId',
 'AccountId',
 'SubscriptionId',
 'CustomerId',
 'CurrencyCode',
 'ProviderId',
 'ProductId',
 'ProductCategory',
 'ChannelId',
 'TransactionStartTime']

In [61]:
convert_to_string(df, string_columns)

In [62]:
df.dtypes

TransactionId           string[python]
BatchId                 string[python]
AccountId               string[python]
SubscriptionId          string[python]
CustomerId              string[python]
CurrencyCode            string[python]
CountryCode                      int64
ProviderId              string[python]
ProductId               string[python]
ProductCategory         string[python]
ChannelId               string[python]
Amount                         float64
Value                            int64
TransactionStartTime    string[python]
PricingStrategy                  int64
FraudResult                      int64
dtype: object

In [63]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [64]:
convert_to_datetime(df, ['TransactionStartTime'])

In [65]:
df.dtypes

TransactionId                string[python]
BatchId                      string[python]
AccountId                    string[python]
SubscriptionId               string[python]
CustomerId                   string[python]
CurrencyCode                 string[python]
CountryCode                           int64
ProviderId                   string[python]
ProductId                    string[python]
ProductCategory              string[python]
ChannelId                    string[python]
Amount                              float64
Value                                 int64
TransactionStartTime    datetime64[ns, UTC]
PricingStrategy                       int64
FraudResult                           int64
dtype: object

In [66]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


## Duplicates

In [67]:
# search for duplicate rows and drop them
drop_duplicates(df)

No duplicate rows were found.


In [68]:
df.duplicated(subset=['TransactionId', 'TransactionStartTime']).all()

False

## Feature Engineering

### Aggregate Features

In [69]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CountryCode,95662.0,256.000000,0.000000,256.0,256.0,256.0,256.0,256.0
Amount,95662.0,6717.846433,123306.797164,-1000000.0,-50.0,1000.0,2800.0,9880000.0
Value,95662.0,9900.583941,123122.087776,2.0,275.0,1000.0,5000.0,9880000.0
PricingStrategy,95662.0,2.255974,0.732924,0.0,2.0,2.0,2.0,4.0
FraudResult,95662.0,0.002018,0.044872,0.0,0.0,0.0,0.0,1.0


In [70]:
df ['TransactionTotal']= df['Value'].sum()
df ['TransactionCount']= df['Value'].count()
df ['TransactionSTD']= df['Value'].std()
df ['Transactionaverage']= df['Value'].mean()

In [71]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage
81362,TransactionId_17742,BatchId_114134,AccountId_4840,SubscriptionId_3829,CustomerId_1922,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-01 13:17:05+00:00,2,0,947109661,95662,123122.087776,9900.583941
27844,TransactionId_37405,BatchId_82901,AccountId_2395,SubscriptionId_1403,CustomerId_2809,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-12-15 03:32:39+00:00,4,0,947109661,95662,123122.087776,9900.583941
19894,TransactionId_126391,BatchId_47265,AccountId_4249,SubscriptionId_4429,CustomerId_7436,UGX,256,ProviderId_4,ProductId_3,airtime,ChannelId_2,-500.0,500,2018-12-07 15:49:59+00:00,4,0,947109661,95662,123122.087776,9900.583941
45762,TransactionId_89476,BatchId_10363,AccountId_2495,SubscriptionId_1733,CustomerId_2911,UGX,256,ProviderId_6,ProductId_2,data_bundles,ChannelId_3,500.0,500,2018-12-29 05:29:19+00:00,2,0,947109661,95662,123122.087776,9900.583941
21182,TransactionId_13530,BatchId_125300,AccountId_768,SubscriptionId_2916,CustomerId_1113,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,1000.0,1000,2018-12-08 18:13:44+00:00,2,0,947109661,95662,123122.087776,9900.583941
72255,TransactionId_91365,BatchId_121388,AccountId_4841,SubscriptionId_3829,CustomerId_3064,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-01-25 13:43:11+00:00,2,0,947109661,95662,123122.087776,9900.583941
16695,TransactionId_36143,BatchId_67879,AccountId_4249,SubscriptionId_4429,CustomerId_7343,UGX,256,ProviderId_4,ProductId_10,airtime,ChannelId_2,-25000.0,25000,2018-12-04 17:09:41+00:00,4,0,947109661,95662,123122.087776,9900.583941
8131,TransactionId_81602,BatchId_48042,AccountId_4213,SubscriptionId_1829,CustomerId_4666,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-25 12:33:09+00:00,4,0,947109661,95662,123122.087776,9900.583941
60997,TransactionId_9161,BatchId_105475,AccountId_1538,SubscriptionId_1972,CustomerId_1911,UGX,256,ProviderId_3,ProductId_7,tv,ChannelId_1,280000.0,288400,2019-01-15 14:15:45+00:00,1,0,947109661,95662,123122.087776,9900.583941
53897,TransactionId_38253,BatchId_138297,AccountId_4249,SubscriptionId_4429,CustomerId_7343,UGX,256,ProviderId_4,ProductId_3,airtime,ChannelId_2,-25000.0,25000,2019-01-08 11:23:50+00:00,4,0,947109661,95662,123122.087776,9900.583941


In [72]:
df['TransactionYear'] = df['TransactionStartTime'].apply(lambda x: x.year)
df['TransactionMonth'] = df['TransactionStartTime'].apply(lambda x: x.month)
df['TransactionDay'] = df['TransactionStartTime'].apply(lambda x: x.day)
df['TransactionHour'] = df['TransactionStartTime'].apply(lambda x: x.hour)

In [73]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
66240,TransactionId_78800,BatchId_59094,AccountId_2990,SubscriptionId_216,CustomerId_3415,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2019-01-20 14:20:34+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,20,14
67788,TransactionId_72897,BatchId_10568,AccountId_4841,SubscriptionId_3829,CustomerId_4840,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-01-22 07:35:26+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,22,7
79084,TransactionId_115692,BatchId_71339,AccountId_4841,SubscriptionId_3829,CustomerId_3292,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-01-31 10:20:44+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,31,10
63348,TransactionId_78475,BatchId_24967,AccountId_4841,SubscriptionId_3829,CustomerId_2876,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-75.0,75,2019-01-18 09:58:16+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,18,9
30350,TransactionId_90832,BatchId_74652,AccountId_4841,SubscriptionId_3829,CustomerId_647,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2018-12-17 07:53:49+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,17,7
81653,TransactionId_70016,BatchId_68061,AccountId_4841,SubscriptionId_3829,CustomerId_3420,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-02-01 14:20:41+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,1,14
88070,TransactionId_1016,BatchId_46231,AccountId_4706,SubscriptionId_3090,CustomerId_5174,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-07 17:02:04+00:00,4,0,947109661,95662,123122.087776,9900.583941,2019,2,7,17
10544,TransactionId_122434,BatchId_69424,AccountId_4100,SubscriptionId_493,CustomerId_4550,UGX,256,ProviderId_3,ProductId_10,airtime,ChannelId_3,500.0,500,2018-11-29 09:03:10+00:00,1,0,947109661,95662,123122.087776,9900.583941,2018,11,29,9
89730,TransactionId_26857,BatchId_9243,AccountId_4841,SubscriptionId_3829,CustomerId_5281,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-02-08 11:48:44+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,8,11
33965,TransactionId_96951,BatchId_37363,AccountId_3017,SubscriptionId_4580,CustomerId_3442,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,3500.0,3500,2018-12-20 16:57:40+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,20,16


In [74]:
df.shape

(95662, 24)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   TransactionId         95662 non-null  string             
 1   BatchId               95662 non-null  string             
 2   AccountId             95662 non-null  string             
 3   SubscriptionId        95662 non-null  string             
 4   CustomerId            95662 non-null  string             
 5   CurrencyCode          95662 non-null  string             
 6   CountryCode           95662 non-null  int64              
 7   ProviderId            95662 non-null  string             
 8   ProductId             95662 non-null  string             
 9   ProductCategory       95662 non-null  string             
 10  ChannelId             95662 non-null  string             
 11  Amount                95662 non-null  float64            
 12  Valu

## One-Hot Encoding 

In [76]:
encoder= ce.OrdinalEncoder(cols=['ProviderId'],return_df=True,
                           mapping=[{'col':'ProviderId',
'mapping':{'None':0,'ProviderId_1':1,'ProviderId_2':2,'ProviderId_3':3,'ProviderId_4':4,'ProviderId_5':5,'ProviderId_6':6}}])

In [77]:
df = encoder.fit_transform(df)

In [78]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
3454,TransactionId_66240,BatchId_27069,AccountId_2237,SubscriptionId_872,CustomerId_2649,UGX,256,3,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-19 11:49:38+00:00,1,0,947109661,95662,123122.087776,9900.583941,2018,11,19,11
57448,TransactionId_22359,BatchId_50090,AccountId_1180,SubscriptionId_400,CustomerId_1537,UGX,256,1,ProductId_15,financial_services,ChannelId_3,5000.0,5000,2019-01-11 15:14:26+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,11,15
40579,TransactionId_33772,BatchId_11430,AccountId_4841,SubscriptionId_3829,CustomerId_444,UGX,256,4,ProductId_6,financial_services,ChannelId_2,-100.0,100,2018-12-26 02:58:01+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,26,2
31937,TransactionId_50452,BatchId_130316,AccountId_4841,SubscriptionId_3829,CustomerId_4722,UGX,256,4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2018-12-18 16:11:25+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,18,16
88904,TransactionId_65934,BatchId_125599,AccountId_4841,SubscriptionId_3829,CustomerId_2845,UGX,256,4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-02-08 07:33:43+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,8,7
58698,TransactionId_44646,BatchId_45300,AccountId_2946,SubscriptionId_3991,CustomerId_3371,UGX,256,6,ProductId_2,data_bundles,ChannelId_3,1750.0,1750,2019-01-12 16:49:17+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,12,16
92403,TransactionId_104789,BatchId_85022,AccountId_3427,SubscriptionId_343,CustomerId_3862,UGX,256,5,ProductId_3,airtime,ChannelId_3,500.0,500,2019-02-10 10:01:43+00:00,4,0,947109661,95662,123122.087776,9900.583941,2019,2,10,10
40064,TransactionId_93865,BatchId_79783,AccountId_986,SubscriptionId_2225,CustomerId_1335,UGX,256,1,ProductId_15,financial_services,ChannelId_3,11500.0,11500,2018-12-25 11:11:12+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,25,11
30362,TransactionId_33003,BatchId_37918,AccountId_4841,SubscriptionId_3829,CustomerId_4204,UGX,256,4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2018-12-17 07:58:41+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,17,7
29819,TransactionId_52463,BatchId_30161,AccountId_3206,SubscriptionId_1372,CustomerId_3634,UGX,256,6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-12-16 17:52:51+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,16,17


In [79]:
# unique value counts
counts_df = df['ProductId'].value_counts()
counts_df

ProductId
ProductId_6     32635
ProductId_3     24344
ProductId_10    15384
ProductId_15    11964
ProductId_1      2885
ProductId_4      2403
ProductId_21     1512
ProductId_11     1377
ProductId_19      965
ProductId_14      789
ProductId_13      296
ProductId_2       236
ProductId_20      216
ProductId_8       197
ProductId_24      175
ProductId_22      112
ProductId_7        86
ProductId_27       31
ProductId_5        25
ProductId_9        17
ProductId_16       10
ProductId_12        2
ProductId_23        1
Name: count, dtype: Int64

In [80]:
encoder= ce.OrdinalEncoder(cols=['ProductId'],return_df=True,
                           mapping=[{'col':'ProductId',
'mapping':{'None':0,'ProductId_6':6,'ProductId_3':3,'ProductId_10':10,'ProductId_15':15,'ProductId_1':1,'ProductId_4':4,'ProductId_21':21,'ProductId_11':11,'ProductId_19':19,'ProductId_14':14,'ProductId_13':13,'ProductId_2':2,'ProductId_20':20,'ProductId_8':8,'ProductId_24':24,'ProductId_22':22,'ProductId_7':7,'ProductId_27':27,'ProductId_5':5,'ProductId_9':9,'ProductId_16':16,'ProductId_12':12,'ProductId_23':23}}])

In [81]:
df = encoder.fit_transform(df)

In [82]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
39387,TransactionId_104246,BatchId_41608,AccountId_3206,SubscriptionId_1372,CustomerId_3634,UGX,256,6,3,airtime,ChannelId_3,1000.0,1000,2018-12-24 18:17:20+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,24,18
8200,TransactionId_48817,BatchId_25706,AccountId_4841,SubscriptionId_3829,CustomerId_2739,UGX,256,4,6,financial_services,ChannelId_2,-622.0,622,2018-11-25 14:38:23+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,11,25,14
3952,TransactionId_53625,BatchId_34015,AccountId_4841,SubscriptionId_3829,CustomerId_349,UGX,256,4,6,financial_services,ChannelId_2,-250.0,250,2018-11-19 16:03:42+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,11,19,16
75003,TransactionId_56171,BatchId_472,AccountId_148,SubscriptionId_3236,CustomerId_471,UGX,256,1,15,financial_services,ChannelId_3,30000.0,30000,2019-01-27 13:21:56+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,27,13
94844,TransactionId_33084,BatchId_76170,AccountId_2157,SubscriptionId_1230,CustomerId_2562,UGX,256,3,15,financial_services,ChannelId_3,2000.0,2000,2019-02-12 14:34:40+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,12,14
51761,TransactionId_53013,BatchId_18453,AccountId_2324,SubscriptionId_1973,CustomerId_2738,UGX,256,5,15,financial_services,ChannelId_3,5000.0,5000,2019-01-06 16:49:05+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,6,16
29344,TransactionId_22103,BatchId_52066,AccountId_4841,SubscriptionId_3829,CustomerId_3634,UGX,256,4,6,financial_services,ChannelId_2,-20.0,20,2018-12-16 12:03:02+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,16,12
64779,TransactionId_93334,BatchId_50876,AccountId_2321,SubscriptionId_2755,CustomerId_2735,UGX,256,5,10,airtime,ChannelId_3,500.0,500,2019-01-18 17:11:23+00:00,4,0,947109661,95662,123122.087776,9900.583941,2019,1,18,17
81095,TransactionId_88470,BatchId_133317,AccountId_4182,SubscriptionId_3248,CustomerId_4634,UGX,256,6,3,airtime,ChannelId_3,5000.0,5000,2019-02-01 11:56:24+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,1,11
62286,TransactionId_2497,BatchId_44336,AccountId_4376,SubscriptionId_3447,CustomerId_4833,UGX,256,5,15,financial_services,ChannelId_3,1942.0,1942,2019-01-17 02:20:59+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,17,2


In [83]:
# unique value counts
counts_df = df['ChannelId'].value_counts()
counts_df

ChannelId
ChannelId_3    56935
ChannelId_2    37141
ChannelId_5     1048
ChannelId_1      538
Name: count, dtype: Int64

In [84]:
encoder= ce.OrdinalEncoder(cols=['ChannelId'],return_df=True,
                           mapping=[{'col':'ChannelId',
'mapping':{'None':0,'ChannelId_1':1,'ChannelId_2':2,'ChannelId_3':3,'ChannelId_5':5}}])

In [85]:
df = encoder.fit_transform(df)

In [86]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
40665,TransactionId_81217,BatchId_30010,AccountId_1670,SubscriptionId_2569,CustomerId_2052,UGX,256,6,3,airtime,3,1000.0,1000,2018-12-26 06:39:57+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,26,6
24878,TransactionId_134426,BatchId_67188,AccountId_4840,SubscriptionId_3829,CustomerId_592,UGX,256,4,6,financial_services,2,-1000.0,1000,2018-12-13 09:49:52+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,13,9
21967,TransactionId_47894,BatchId_62892,AccountId_3966,SubscriptionId_1094,CustomerId_4416,UGX,256,1,10,airtime,3,4000.0,4000,2018-12-10 06:30:26+00:00,4,0,947109661,95662,123122.087776,9900.583941,2018,12,10,6
80309,TransactionId_77159,BatchId_54648,AccountId_4841,SubscriptionId_3829,CustomerId_2145,UGX,256,4,6,financial_services,2,-5000.0,5000,2019-02-01 07:43:11+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,1,7
89737,TransactionId_121074,BatchId_9799,AccountId_4841,SubscriptionId_3829,CustomerId_3180,UGX,256,4,6,financial_services,2,-5000.0,5000,2019-02-08 11:49:43+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,8,11
11244,TransactionId_41776,BatchId_23708,AccountId_4841,SubscriptionId_3829,CustomerId_453,UGX,256,4,6,financial_services,2,-5000.0,5000,2018-11-30 08:11:18+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,11,30,8
38840,TransactionId_3117,BatchId_36560,AccountId_3071,SubscriptionId_2757,CustomerId_3497,UGX,256,5,15,financial_services,3,1000.0,1000,2018-12-24 10:50:05+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,24,10
51309,TransactionId_45720,BatchId_62872,AccountId_318,SubscriptionId_3087,CustomerId_647,UGX,256,6,10,airtime,3,1000.0,1000,2019-01-05 15:19:10+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,5,15
29178,TransactionId_48510,BatchId_97750,AccountId_3533,SubscriptionId_328,CustomerId_3970,UGX,256,6,3,airtime,3,2000.0,2000,2018-12-16 09:45:09+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,16,9
76190,TransactionId_28137,BatchId_44218,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,4,6,financial_services,2,-50.0,50,2019-01-28 12:21:33+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,28,12


In [88]:
# unique value counts
counts_df = df['CurrencyCode'].value_counts()
counts_df

CurrencyCode
UGX    95662
Name: count, dtype: Int64

In [89]:
encoder= ce.OrdinalEncoder(cols=['CurrencyCode'],return_df=True,
                           mapping=[{'col':'CurrencyCode',
'mapping':{'None':0,'UGX':1}}])

In [90]:
df = encoder.fit_transform(df)

In [91]:
df.sample(10)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTotal,TransactionCount,TransactionSTD,Transactionaverage,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
41088,TransactionId_21597,BatchId_113369,AccountId_4841,SubscriptionId_3829,CustomerId_3066,1,256,4,6,financial_services,2,-20.0,20,2018-12-26 15:32:39+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,26,15
36426,TransactionId_95334,BatchId_32379,AccountId_2124,SubscriptionId_4735,CustomerId_2529,1,256,6,3,airtime,3,500.0,500,2018-12-21 19:45:47+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,21,19
5013,TransactionId_45898,BatchId_132461,AccountId_4018,SubscriptionId_316,CustomerId_4468,1,256,5,3,airtime,3,1000.0,1000,2018-11-21 11:59:23+00:00,4,0,947109661,95662,123122.087776,9900.583941,2018,11,21,11
23104,TransactionId_85329,BatchId_67754,AccountId_2292,SubscriptionId_1248,CustomerId_2706,1,256,6,3,airtime,3,1800.0,1800,2018-12-11 12:35:48+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,11,12
1600,TransactionId_54510,BatchId_55016,AccountId_2759,SubscriptionId_3781,CustomerId_3180,1,256,6,3,airtime,3,1000.0,1000,2018-11-16 13:45:22+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,11,16,13
44403,TransactionId_86096,BatchId_84431,AccountId_3071,SubscriptionId_2757,CustomerId_3497,1,256,5,15,financial_services,3,1000.0,1000,2018-12-28 13:25:43+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,28,13
33083,TransactionId_78917,BatchId_80037,AccountId_4421,SubscriptionId_4038,CustomerId_4878,1,256,5,15,financial_services,3,195000.0,195000,2018-12-19 18:19:03+00:00,2,0,947109661,95662,123122.087776,9900.583941,2018,12,19,18
94212,TransactionId_21100,BatchId_9193,AccountId_4841,SubscriptionId_3829,CustomerId_682,1,256,4,6,financial_services,2,-25.0,25,2019-02-12 04:34:25+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,2,12,4
61877,TransactionId_94507,BatchId_71388,AccountId_4841,SubscriptionId_3829,CustomerId_1956,1,256,4,6,financial_services,2,-50.0,50,2019-01-16 14:52:02+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,16,14
63466,TransactionId_97365,BatchId_40807,AccountId_4841,SubscriptionId_3829,CustomerId_1707,1,256,4,6,financial_services,2,-250.0,250,2019-01-18 10:07:46+00:00,2,0,947109661,95662,123122.087776,9900.583941,2019,1,18,10


In [99]:
# unique value counts
counts_df = df['ProductCategory'].value_counts()
counts_df

ProductCategory
financial_services    45405
airtime               45027
utility_bill           1920
data_bundles           1613
tv                     1279
ticket                  216
movies                  175
transport                25
other                     2
Name: count, dtype: Int64

# Univariate Analysis

## Non-Graphical Univariate Analysis